In [1]:
import numpy as np

tau_relax = 27
delta_tau = 5
P = 23
eta = np.random.uniform(0.01, 0.5)

# The direct path tau_p = 0 
tau_p = [0]

# Generate remaining delays tau_p using uniform distribution, excluding zero, allowing repeats
remaining_tau_p = np.random.randint(1, tau_relax - delta_tau + 1, P - 1)

# Combine and sort 
tau_p = np.concatenate((tau_p, remaining_tau_p))
tau_p = np.sort(tau_p)

# Calculate amplitude gains alpha_p of h_tau_p
alpha_p = np.exp(-eta * tau_p / 2)

# Generate random phases phi_p
phi_p = np.random.uniform(0, 2 * np.pi, P)

# Calculate complex gains h_tau_p
h_tau_p = alpha_p * np.exp(1j * phi_p)

for p in range(P):
    print(f"Path {p + 1}: tau_p = {tau_p[p]}, alpha_p = {alpha_p[p]:.4f}, phi_p = {phi_p[p]:.4f}, h_tau_p = {h_tau_p[p]:.4f}")


Path 1: tau_p = 0, alpha_p = 1.0000, phi_p = 6.2424, h_tau_p = 0.9992-0.0408j
Path 2: tau_p = 2, alpha_p = 0.7165, phi_p = 5.6255, h_tau_p = 0.5671-0.4380j
Path 3: tau_p = 3, alpha_p = 0.6065, phi_p = 6.1479, h_tau_p = 0.6010-0.0818j
Path 4: tau_p = 4, alpha_p = 0.5134, phi_p = 2.4698, h_tau_p = -0.4019+0.3195j
Path 5: tau_p = 4, alpha_p = 0.5134, phi_p = 3.0759, h_tau_p = -0.5123+0.0337j
Path 6: tau_p = 7, alpha_p = 0.3114, phi_p = 0.3936, h_tau_p = 0.2876+0.1194j
Path 7: tau_p = 9, alpha_p = 0.2231, phi_p = 0.2203, h_tau_p = 0.2178+0.0488j
Path 8: tau_p = 11, alpha_p = 0.1599, phi_p = 4.0531, h_tau_p = -0.0979-0.1264j
Path 9: tau_p = 11, alpha_p = 0.1599, phi_p = 4.5297, h_tau_p = -0.0290-0.1572j
Path 10: tau_p = 12, alpha_p = 0.1353, phi_p = 0.0294, h_tau_p = 0.1353+0.0040j
Path 11: tau_p = 13, alpha_p = 0.1146, phi_p = 2.6159, h_tau_p = -0.0991+0.0575j
Path 12: tau_p = 13, alpha_p = 0.1146, phi_p = 3.5269, h_tau_p = -0.1062-0.0431j
Path 13: tau_p = 14, alpha_p = 0.0970, phi_p = 1.3